In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd  /content/drive/MyDrive/Capstone2/
!ls


/content/drive/MyDrive/Capstone2
class-descriptions-boxable.csv	train-annotations-bbox.csv
image				train-images-boxable-with-rotation.csv
Inception.ipynb			train_process.ipynb
model				Untitled0.ipynb
preprocess_image.ipynb		Untitled1.ipynb
test


In [3]:
!pip install tensorflow == 2.0

ERROR: Invalid requirement: '=='


In [4]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from skimage import io
from shutil import copyfile
import sys
import time
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
!wget https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv


--2021-04-12 19:56:38--  https://storage.googleapis.com/openimages/2018_04/train/train-annotations-bbox.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.128, 173.194.211.128, 173.194.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1194033454 (1.1G) [text/csv]
Saving to: ‘train-annotations-bbox.csv.1’

train-annotations-b 100%[===================>]   1.11G  36.9MB/s    in 37s     

2021-04-12 19:57:16 (30.5 MB/s) - ‘train-annotations-bbox.csv.1’ saved [1194033454/1194033454]



In [6]:
!wget https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv

--2021-04-12 19:57:16--  https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 638407721 (609M) [text/csv]
Saving to: ‘train-images-boxable-with-rotation.csv.1’

train-images-boxabl 100%[===================>] 608.83M  30.7MB/s    in 20s     

2021-04-12 19:57:36 (30.3 MB/s) - ‘train-images-boxable-with-rotation.csv.1’ saved [638407721/638407721]



In [7]:
!wget https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv


--2021-04-12 19:57:36--  https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 173.194.213.128, 173.194.215.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12011 (12K) [text/csv]
Saving to: ‘class-descriptions-boxable.csv.1’

class-descriptions- 100%[===================>]  11.73K  --.-KB/s    in 0.003s  

2021-04-12 19:57:36 (4.04 MB/s) - ‘class-descriptions-boxable.csv.1’ saved [12011/12011]



In [8]:
images_boxable_fname = 'train-images-boxable-with-rotation.csv'
annotations_bbox_fname = 'train-annotations-bbox.csv'
class_descriptions_fname = 'class-descriptions-boxable.csv'

In [9]:
images_boxable = pd.read_csv(images_boxable_fname)
images_boxable.head()

,ImageID,Subset,OriginalURL,OriginalLandingURL,License,AuthorProfileURL,Author,Title,OriginalSize,OriginalMD5,Thumbnail300KURL,Rotation
0,4fa8054781a4c382,train,https://farm3.staticflickr.com/5310/5898076654...,https://www.flickr.com/photos/michael-beat/589...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/michael-beat/,Michael Beat,...die FNF-Kerze,4405052,KFukvivpCM5QXl5SqKe41g==,https://c1.staticflickr.com/6/5310/5898076654_...,0.0
1,b37f763ae67d0888,train,https://c1.staticflickr.com/1/67/197493648_628...,https://www.flickr.com/photos/drstarbuck/19749...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/drstarbuck/,Karen,Three boys on a hill,494555,9IzEn38GRNsVpATuv7gzEA==,https://c3.staticflickr.com/1/67/197493648_628...,0.0
2,7e8584b0f487cb9e,train,https://c7.staticflickr.com/8/7056/7143870979_...,https://www.flickr.com/photos/circasassy/71438...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/circasassy/,CircaSassy,A Christmas carol and The cricket on the heart...,2371584,3hQwu0iSzY1VIoXiwp0/Mg==,https://c7.staticflickr.com/8/7056/7143870979_...,0.0
3,86638230febe21c4,train,https://farm5.staticflickr.com/5128/5301868579...,https://www.flickr.com/photos/ajcreencia/53018...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/ajcreencia/,Alex,Abbey and Kenny,949267,onB+rCZnGQg5PRX7xOs18Q==,https://c4.staticflickr.com/6/5128/5301868579_...,NaN
4,249086e72671397d,train,https://c6.staticflickr.com/4/3930/15342460029...,https://www.flickr.com/photos/codnewsroom/1534...,https://creativecommons.org/licenses/by/2.0/,https://www.flickr.com/people/codnewsroom/,COD Newsroom,Suburban Law Enforcement Academy 20th Annivers...,6541758,MjpaAVbMAWbCusSaxI1D7w==,https://c1.staticflickr.com/4/3930/15342460029...,0.0


In [10]:
annotations_bbox = pd.read_csv(annotations_bbox_fname)
annotations_bbox.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,000002b66c9c498e,xclick,/m/01g317,1,0.012500,0.195312,0.148438,0.587500,0,1,0,0,0
1,000002b66c9c498e,xclick,/m/01g317,1,0.025000,0.276563,0.714063,0.948438,0,1,0,0,0
2,000002b66c9c498e,xclick,/m/01g317,1,0.151562,0.310937,0.198437,0.590625,1,0,0,0,0
3,000002b66c9c498e,xclick,/m/01g317,1,0.256250,0.429688,0.651563,0.925000,1,0,0,0,0
4,000002b66c9c498e,xclick,/m/01g317,1,0.257812,0.346875,0.235938,0.385938,1,0,0,0,0


In [11]:
class_descriptions = pd.read_csv(class_descriptions_fname, header=None)
class_descriptions.head()

,0,1
0,/m/011k07,Tortoise
1,/m/011q46kg,Container
2,/m/012074,Magpie
3,/m/0120dh,Sea turtle
4,/m/01226z,Football


In [12]:
def plot_bbox(img_id):
  img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0]
  img = io.imread(img_url)
  height, width, channel = img.shape
  print(f"Image: {img.shape}")
  bboxs = annotations_bbox[annotations_bbox['ImageID']==img_id]
  for index, row in bboxs.iterrows():
      xmin = row['XMin']
      xmax = row['XMax']
      ymin = row['YMin']
      ymax = row['YMax']
      xmin = int(xmin*width)
      xmax = int(xmax*width)
      ymin = int(ymin*height)
      ymax = int(ymax*height)
      label_name = row['LabelName']
      class_series = class_descriptions[class_descriptions[0]==label_name]
      class_name = class_series[1].values[0]
      print(f"Coordinates: {xmin,ymin}, {xmax,ymax}")
      cv2.rectangle(img, (xmin,ymin), (xmax,ymax), (255,0,0), 5)
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(img, class_name, (xmin,ymin-10), font, 3, (0,255,0), 5)
  plt.figure(figsize=(15,10))
  plt.title('Image with Bounding Box')
  plt.imshow(img)
  plt.axis("off")
  plt.show()

In [13]:
least_objects_img_ids = annotations_bbox["ImageID"].value_counts().tail(50).index.values
for img_id in random.sample(list(least_objects_img_ids), 5):
  try:
    plot_bbox(img_id)
  except Exception:
    print('Not found URL')

Output hidden; open in https://colab.research.google.com to view.

In [14]:
class_descriptions.loc[class_descriptions[1].isin(['Person', 'Oven'])]

,0,1
68,/m/01g317,Person
150,/m/029bxz,Oven


In [15]:
person_pd = class_descriptions[class_descriptions[1]=='Person']
oven_pd = class_descriptions[class_descriptions[1]=='Oven']
label_name_person = person_pd[0].values[0]
label_name_oven = oven_pd[0].values[0]

In [16]:
person_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_person]
oven_bbox = annotations_bbox[annotations_bbox['LabelName']==label_name_oven]

In [17]:
print('There are %d persons in the dataset' %(len(person_bbox)))
print('There are %d ovens in the dataset' %(len(oven_bbox)))


There are 1034721 persons in the dataset
There are 637 ovens in the dataset


In [18]:
image_path = 'image'

In [19]:
def crawl_data_to(dataframe):
  if not os.path.exists(image_path):
    os.mkdir(image_path)
  i = 0
  n = 637
  n_issues = 0
  
  label = class_descriptions[class_descriptions[0]==dataframe.iloc[0]['LabelName']][1].values[0]
  print(label)
  for index,row in dataframe.sample(n).iterrows():
    i+=1
    print('[INFO] Crawling {}/{} with label {}'.format(i,n,label))
    try:
      img_id = row['ImageID']
      img_url = images_boxable.loc[images_boxable["ImageID"]==img_id]['OriginalURL'].values[0] 
      img = io.imread(img_url)
      height, width, channel = img.shape
      xmin = row['XMin']
      xmax = row['XMax']
      ymin = row['YMin']
      ymax = row['YMax']
      xmin = int(xmin*width)
      xmax = int(xmax*width)
      ymin = int(ymin*height)
      ymax = int(ymax*height) 
      img = img[ymin:ymax,xmin:xmax]  
      save_path = os.path.join(image_path,label+'-'+img_id+'.jpg')
      io.imsave(save_path,img)
    except Exception as e:
      n_issues +=1
  print(f"Images Issues: {n_issues} with {label}")


In [20]:
crawl_data_to(oven_bbox)

Oven
[INFO] Crawling 1/637 with label Oven
[INFO] Crawling 2/637 with label Oven
[INFO] Crawling 3/637 with label Oven
[INFO] Crawling 4/637 with label Oven
[INFO] Crawling 5/637 with label Oven
[INFO] Crawling 6/637 with label Oven
[INFO] Crawling 7/637 with label Oven
[INFO] Crawling 8/637 with label Oven
[INFO] Crawling 9/637 with label Oven
[INFO] Crawling 10/637 with label Oven
[INFO] Crawling 11/637 with label Oven
[INFO] Crawling 12/637 with label Oven
[INFO] Crawling 13/637 with label Oven
[INFO] Crawling 14/637 with label Oven
[INFO] Crawling 15/637 with label Oven
[INFO] Crawling 16/637 with label Oven
[INFO] Crawling 17/637 with label Oven
[INFO] Crawling 18/637 with label Oven
[INFO] Crawling 19/637 with label Oven
[INFO] Crawling 20/637 with label Oven
[INFO] Crawling 21/637 with label Oven
[INFO] Crawling 22/637 with label Oven
[INFO] Crawling 23/637 with label Oven
[INFO] Crawling 24/637 with label Oven
[INFO] Crawling 25/637 with label Oven
[INFO] Crawling 26/637 with l

In [21]:
crawl_data_to(person_bbox)

Person
[INFO] Crawling 1/637 with label Person
[INFO] Crawling 2/637 with label Person
[INFO] Crawling 3/637 with label Person
[INFO] Crawling 4/637 with label Person
[INFO] Crawling 5/637 with label Person
[INFO] Crawling 6/637 with label Person
[INFO] Crawling 7/637 with label Person
[INFO] Crawling 8/637 with label Person
[INFO] Crawling 9/637 with label Person
[INFO] Crawling 10/637 with label Person
[INFO] Crawling 11/637 with label Person
[INFO] Crawling 12/637 with label Person
[INFO] Crawling 13/637 with label Person
[INFO] Crawling 14/637 with label Person
[INFO] Crawling 15/637 with label Person
[INFO] Crawling 16/637 with label Person
[INFO] Crawling 17/637 with label Person
[INFO] Crawling 18/637 with label Person
[INFO] Crawling 19/637 with label Person
[INFO] Crawling 20/637 with label Person
[INFO] Crawling 21/637 with label Person
[INFO] Crawling 22/637 with label Person
[INFO] Crawling 23/637 with label Person
[INFO] Crawling 24/637 with label Person
[INFO] Crawling 25